In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import re
import lightgbm as lgb
import datetime as dt
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

In [2]:
borrar = ["Opportunity_Name", "ID", "Brand", "Product_Type", "ASP_converted_Currency", 
          "Prod_Category_A", "Product_Category_B", "Actual_Delivery_Date", "Last_Activity", 
          "ASP_converted_Currency", "Prod_Category_A", "Product_Name", "Delivery_Year", "Month", 
          "TRF", "Product_Family", "Account_Name"]

useless = ["Submitted_for_Approval", "Account_Type", "Delivery_Terms", "Size", "Price", "ASP_Currency", 
           "Total_Amount_Currency", "Total_Taxable_Amount_Currency","Quote_Type", "Opportunity_Type"] # "Product_Category_B", "Region"]

dates = ["Account_Created_Date", "Opportunity_Created_Date", "Quote_Expiry_Date", "Last_Modified_Date", 
         "Planned_Delivery_Start_Date", "Planned_Delivery_End_Date"]

target = ["Opportunity_ID", "Stage", "Sales_Contract_No"]

In [3]:
def preprocess(data):
    
    # elimino caracteres prohibidos en los headers
    
    data = data.rename(columns = lambda x:re.sub("[^A-Za-z0-9_]+", "", x))
        
    for d in dates:
        data[d] = pd.to_datetime(data[d])

    # agrego features
    
    data["Contacts"] = data.groupby("Opportunity_ID", sort = False)["Opportunity_ID"].transform("count")
    data["Delivery_Difference"] = (data["Planned_Delivery_End_Date"] - data["Planned_Delivery_Start_Date"]).dt.days
    data["Same_Owner"] = (data.Opportunity_Owner == data.Account_Owner) & (data.Opportunity_Owner == data.Last_Modified_By)
    data["Has_Brand"] = data.Brand != "None"
    data["Has_Contract"] = data.Sales_Contract_No != "None"
    data["Different_Country"] = (data.Billing_Country != data.Territory) & (data.Territory != "None")    
    data.loc[data.TRF == 0, "TRF_Cat"] = 0
    data.loc[(1 <= data.TRF) & (data.TRF <= 7), "TRF_Cat"] = 1
    data.loc[data.TRF > 7, "TRF_Cat"] = 2
    data["Sales"] = data.groupby("Account_Name", sort = False)["Account_Name"].transform("count")
    
    data["Concrete_Offer"] = (data["Planned_Delivery_End_Date"] - data["Opportunity_Created_Date"]).dt.days
    
    data["Offer_Duration"] = (data["Quote_Expiry_Date"] - data["Opportunity_Created_Date"]).dt.days
    
    # fabri
    
    data["Territory_Defined"] = data.Territory != "None"
    data["Past_Quote"] = (data["Last_Modified_Date"] - data["Quote_Expiry_Date"]).dt.days
    data["Month"] = data.Month.str[-2:].astype(int)
    
    # casteo a categoricas varias columnas
    categorical = [x for x in data.columns if data[x].dtype == "object"]
    for c in categorical:       
        data[c] = data[c].astype('category')
        
    # limpio columnas
    
    data = data.drop(borrar + useless + dates + ["Delivery_Quarter"], axis = 1)
    
    #data.drop(data[(data.Contacts > 1) & (data.ASP.isna())].index, inplace = True)
    #data = data.drop_duplicates([x for x in data.columns if x != 'ID'])
    
    return (data)

In [4]:
import catboost as ctb

In [5]:
data = pd.read_csv("Entrenamieto_ECI_2020.csv")

In [6]:
prep = preprocess(data)

In [7]:
prep = prep[(prep.Stage == "Closed Won") | (prep.Stage == "Closed Lost")]
prep.Stage = prep.Stage.replace({"Closed Won": 1, "Closed Lost": 0})

In [8]:
x_train, x_test, y_train, y_test = train_test_split(prep, prep.Stage, test_size=0.3, random_state=0)

In [9]:
cat_vars = [x for x in x_train.drop(target, axis=1).select_dtypes('category').columns if x != 'Stage']

In [10]:
cat_indexes = [x_train.drop(target, axis=1).columns.get_loc(x) for x in cat_vars]

In [11]:
model = ctb.CatBoostClassifier(num_boost_round=5000,
                               learning_rate=0.015,
                               l2_leaf_reg=3.5,
                               depth=8, 
                               max_bin=400,
                               num_leaves=31,
                               rsm=0.98,
                               eval_metric='Logloss',
                               use_best_model=True,
                               random_seed=42)

In [ ]:
model.fit(x_train.drop(target, axis=1), y_train, eval_set=(x_test.drop(target, axis=1), y_test), cat_features=cat_indexes, early_stopping_rounds=800)

0:	learn: 0.6692143	test: 0.6694664	best: 0.6694664 (0)	total: 74.2ms	remaining: 6m 11s
1:	learn: 0.6453116	test: 0.6457598	best: 0.6457598 (1)	total: 95.8ms	remaining: 3m 59s
2:	learn: 0.6220160	test: 0.6229542	best: 0.6229542 (2)	total: 114ms	remaining: 3m 10s
3:	learn: 0.5995892	test: 0.6007760	best: 0.6007760 (3)	total: 127ms	remaining: 2m 38s
4:	learn: 0.5797693	test: 0.5811600	best: 0.5811600 (4)	total: 137ms	remaining: 2m 16s
5:	learn: 0.5618440	test: 0.5634543	best: 0.5634543 (5)	total: 144ms	remaining: 1m 59s
6:	learn: 0.5428110	test: 0.5447323	best: 0.5447323 (6)	total: 159ms	remaining: 1m 53s
7:	learn: 0.5266820	test: 0.5288061	best: 0.5288061 (7)	total: 164ms	remaining: 1m 42s
8:	learn: 0.5095196	test: 0.5117294	best: 0.5117294 (8)	total: 172ms	remaining: 1m 35s
9:	learn: 0.4921522	test: 0.4944084	best: 0.4944084 (9)	total: 188ms	remaining: 1m 33s
10:	learn: 0.4776208	test: 0.4795645	best: 0.4795645 (10)	total: 199ms	remaining: 1m 30s
11:	learn: 0.4627561	test: 0.4651546	be

97:	learn: 0.1208017	test: 0.1314231	best: 0.1314231 (97)	total: 1.54s	remaining: 1m 17s
98:	learn: 0.1202472	test: 0.1309137	best: 0.1309137 (98)	total: 1.56s	remaining: 1m 17s
99:	learn: 0.1195049	test: 0.1302071	best: 0.1302071 (99)	total: 1.58s	remaining: 1m 17s
100:	learn: 0.1187978	test: 0.1295334	best: 0.1295334 (100)	total: 1.59s	remaining: 1m 17s
101:	learn: 0.1181225	test: 0.1289293	best: 0.1289293 (101)	total: 1.61s	remaining: 1m 17s
102:	learn: 0.1175187	test: 0.1283932	best: 0.1283932 (102)	total: 1.63s	remaining: 1m 17s
103:	learn: 0.1169584	test: 0.1278642	best: 0.1278642 (103)	total: 1.65s	remaining: 1m 17s
104:	learn: 0.1163684	test: 0.1273068	best: 0.1273068 (104)	total: 1.66s	remaining: 1m 17s
105:	learn: 0.1158559	test: 0.1268040	best: 0.1268040 (105)	total: 1.68s	remaining: 1m 17s
106:	learn: 0.1153287	test: 0.1263246	best: 0.1263246 (106)	total: 1.7s	remaining: 1m 17s
107:	learn: 0.1147290	test: 0.1257808	best: 0.1257808 (107)	total: 1.71s	remaining: 1m 17s
108:	l

192:	learn: 0.0893927	test: 0.1038582	best: 0.1038582 (192)	total: 3.21s	remaining: 1m 19s
193:	learn: 0.0892277	test: 0.1037034	best: 0.1037034 (193)	total: 3.23s	remaining: 1m 19s
194:	learn: 0.0890040	test: 0.1034925	best: 0.1034925 (194)	total: 3.24s	remaining: 1m 19s
195:	learn: 0.0888603	test: 0.1033386	best: 0.1033386 (195)	total: 3.26s	remaining: 1m 19s
196:	learn: 0.0886873	test: 0.1032203	best: 0.1032203 (196)	total: 3.28s	remaining: 1m 20s
197:	learn: 0.0885728	test: 0.1031370	best: 0.1031370 (197)	total: 3.3s	remaining: 1m 19s
198:	learn: 0.0883360	test: 0.1029628	best: 0.1029628 (198)	total: 3.32s	remaining: 1m 20s
199:	learn: 0.0882264	test: 0.1028729	best: 0.1028729 (199)	total: 3.34s	remaining: 1m 20s
200:	learn: 0.0879070	test: 0.1026169	best: 0.1026169 (200)	total: 3.36s	remaining: 1m 20s
201:	learn: 0.0877705	test: 0.1025250	best: 0.1025250 (201)	total: 3.38s	remaining: 1m 20s
202:	learn: 0.0875023	test: 0.1022861	best: 0.1022861 (202)	total: 3.4s	remaining: 1m 20s
2

288:	learn: 0.0758518	test: 0.0932722	best: 0.0932722 (288)	total: 4.91s	remaining: 1m 20s
289:	learn: 0.0756765	test: 0.0931942	best: 0.0931942 (289)	total: 4.93s	remaining: 1m 20s
290:	learn: 0.0755852	test: 0.0931302	best: 0.0931302 (290)	total: 4.95s	remaining: 1m 20s
291:	learn: 0.0755110	test: 0.0931019	best: 0.0931019 (291)	total: 4.96s	remaining: 1m 20s
292:	learn: 0.0754112	test: 0.0930127	best: 0.0930127 (292)	total: 4.98s	remaining: 1m 20s
293:	learn: 0.0753177	test: 0.0929659	best: 0.0929659 (293)	total: 5s	remaining: 1m 20s
294:	learn: 0.0752369	test: 0.0929128	best: 0.0929128 (294)	total: 5.02s	remaining: 1m 20s
295:	learn: 0.0751576	test: 0.0928501	best: 0.0928501 (295)	total: 5.04s	remaining: 1m 20s
296:	learn: 0.0751222	test: 0.0928111	best: 0.0928111 (296)	total: 5.05s	remaining: 1m 19s
297:	learn: 0.0750327	test: 0.0927389	best: 0.0927389 (297)	total: 5.06s	remaining: 1m 19s
298:	learn: 0.0748721	test: 0.0926350	best: 0.0926350 (298)	total: 5.08s	remaining: 1m 19s
29

389:	learn: 0.0672449	test: 0.0872816	best: 0.0872816 (389)	total: 6.58s	remaining: 1m 17s
390:	learn: 0.0670636	test: 0.0871046	best: 0.0871046 (390)	total: 6.59s	remaining: 1m 17s
391:	learn: 0.0670176	test: 0.0870825	best: 0.0870825 (391)	total: 6.61s	remaining: 1m 17s
392:	learn: 0.0669759	test: 0.0870672	best: 0.0870672 (392)	total: 6.63s	remaining: 1m 17s
393:	learn: 0.0668898	test: 0.0870336	best: 0.0870336 (393)	total: 6.64s	remaining: 1m 17s
394:	learn: 0.0668113	test: 0.0869929	best: 0.0869929 (394)	total: 6.66s	remaining: 1m 17s
395:	learn: 0.0667855	test: 0.0869846	best: 0.0869846 (395)	total: 6.68s	remaining: 1m 17s
396:	learn: 0.0667499	test: 0.0869644	best: 0.0869644 (396)	total: 6.69s	remaining: 1m 17s
397:	learn: 0.0666998	test: 0.0869251	best: 0.0869251 (397)	total: 6.71s	remaining: 1m 17s
398:	learn: 0.0666684	test: 0.0869059	best: 0.0869059 (398)	total: 6.73s	remaining: 1m 17s
399:	learn: 0.0666290	test: 0.0868769	best: 0.0868769 (399)	total: 6.74s	remaining: 1m 17s

...

2680:	learn: 0.0166102	test: 0.0612558	best: 0.0612541 (2679)	total: 48.8s	remaining: 42.2s
2681:	learn: 0.0166062	test: 0.0612671	best: 0.0612541 (2679)	total: 48.9s	remaining: 42.2s
2682:	learn: 0.0166062	test: 0.0612671	best: 0.0612541 (2679)	total: 48.9s	remaining: 42.2s
2683:	learn: 0.0166047	test: 0.0612645	best: 0.0612541 (2679)	total: 48.9s	remaining: 42.2s
2684:	learn: 0.0166017	test: 0.0612714	best: 0.0612541 (2679)	total: 48.9s	remaining: 42.2s
2685:	learn: 0.0166017	test: 0.0612714	best: 0.0612541 (2679)	total: 48.9s	remaining: 42.1s
2686:	learn: 0.0165911	test: 0.0612625	best: 0.0612541 (2679)	total: 48.9s	remaining: 42.1s
2687:	learn: 0.0165861	test: 0.0612568	best: 0.0612541 (2679)	total: 49s	remaining: 42.1s
2688:	learn: 0.0165782	test: 0.0612571	best: 0.0612541 (2679)	total: 49s	remaining: 42.1s
2689:	learn: 0.0165757	test: 0.0612558	best: 0.0612541 (2679)	total: 49s	remaining: 42.1s
2690:	learn: 0.0165757	test: 0.0612558	best: 0.0612541 (2679)	total: 49s	remaining: 42

2777:	learn: 0.0158825	test: 0.0610728	best: 0.0610728 (2777)	total: 50.5s	remaining: 40.4s
2778:	learn: 0.0158790	test: 0.0610787	best: 0.0610728 (2777)	total: 50.5s	remaining: 40.4s
2779:	learn: 0.0158738	test: 0.0610723	best: 0.0610723 (2779)	total: 50.5s	remaining: 40.4s
2780:	learn: 0.0158727	test: 0.0610767	best: 0.0610723 (2779)	total: 50.6s	remaining: 40.3s
2781:	learn: 0.0158727	test: 0.0610767	best: 0.0610723 (2779)	total: 50.6s	remaining: 40.3s
2782:	learn: 0.0158680	test: 0.0610774	best: 0.0610723 (2779)	total: 50.6s	remaining: 40.3s
2783:	learn: 0.0158680	test: 0.0610774	best: 0.0610723 (2779)	total: 50.6s	remaining: 40.3s
2784:	learn: 0.0158680	test: 0.0610774	best: 0.0610723 (2779)	total: 50.6s	remaining: 40.3s
2785:	learn: 0.0158680	test: 0.0610774	best: 0.0610723 (2779)	total: 50.6s	remaining: 40.2s
2786:	learn: 0.0158664	test: 0.0610804	best: 0.0610723 (2779)	total: 50.6s	remaining: 40.2s
2787:	learn: 0.0158543	test: 0.0610690	best: 0.0610690 (2787)	total: 50.7s	remai

2868:	learn: 0.0153556	test: 0.0610106	best: 0.0609995 (2866)	total: 52.2s	remaining: 38.8s
2869:	learn: 0.0153556	test: 0.0610108	best: 0.0609995 (2866)	total: 52.2s	remaining: 38.7s
2870:	learn: 0.0153556	test: 0.0610108	best: 0.0609995 (2866)	total: 52.2s	remaining: 38.7s
2871:	learn: 0.0153435	test: 0.0609979	best: 0.0609979 (2871)	total: 52.2s	remaining: 38.7s
2872:	learn: 0.0153305	test: 0.0609933	best: 0.0609933 (2872)	total: 52.2s	remaining: 38.7s
2873:	learn: 0.0153305	test: 0.0609933	best: 0.0609933 (2872)	total: 52.3s	remaining: 38.7s
2874:	learn: 0.0153305	test: 0.0609933	best: 0.0609933 (2874)	total: 52.3s	remaining: 38.6s
2875:	learn: 0.0153096	test: 0.0609913	best: 0.0609913 (2875)	total: 52.3s	remaining: 38.6s
2876:	learn: 0.0152969	test: 0.0609869	best: 0.0609869 (2876)	total: 52.3s	remaining: 38.6s
2877:	learn: 0.0152844	test: 0.0609706	best: 0.0609706 (2877)	total: 52.3s	remaining: 38.6s
2878:	learn: 0.0152825	test: 0.0609708	best: 0.0609706 (2877)	total: 52.4s	remai

2958:	learn: 0.0147517	test: 0.0608260	best: 0.0608260 (2958)	total: 53.9s	remaining: 37.1s
2959:	learn: 0.0147481	test: 0.0608178	best: 0.0608178 (2959)	total: 53.9s	remaining: 37.1s
2960:	learn: 0.0147443	test: 0.0608258	best: 0.0608178 (2959)	total: 53.9s	remaining: 37.1s
2961:	learn: 0.0147443	test: 0.0608258	best: 0.0608178 (2959)	total: 53.9s	remaining: 37.1s
2962:	learn: 0.0147426	test: 0.0608331	best: 0.0608178 (2959)	total: 53.9s	remaining: 37.1s
2963:	learn: 0.0147331	test: 0.0608350	best: 0.0608178 (2959)	total: 53.9s	remaining: 37.1s
2964:	learn: 0.0147259	test: 0.0608274	best: 0.0608178 (2959)	total: 54s	remaining: 37s
2965:	learn: 0.0147148	test: 0.0608098	best: 0.0608098 (2965)	total: 54s	remaining: 37s
2966:	learn: 0.0147096	test: 0.0608030	best: 0.0608030 (2966)	total: 54s	remaining: 37s
2967:	learn: 0.0147055	test: 0.0608070	best: 0.0608030 (2966)	total: 54s	remaining: 37s
2968:	learn: 0.0147025	test: 0.0608049	best: 0.0608030 (2966)	total: 54s	remaining: 37s
2969:	le

3053:	learn: 0.0142685	test: 0.0607152	best: 0.0607152 (3053)	total: 55.5s	remaining: 35.4s
3054:	learn: 0.0142644	test: 0.0607176	best: 0.0607152 (3053)	total: 55.5s	remaining: 35.4s
3055:	learn: 0.0142605	test: 0.0607193	best: 0.0607152 (3053)	total: 55.6s	remaining: 35.3s
3056:	learn: 0.0142529	test: 0.0607209	best: 0.0607152 (3053)	total: 55.6s	remaining: 35.3s
3057:	learn: 0.0142501	test: 0.0607193	best: 0.0607152 (3053)	total: 55.6s	remaining: 35.3s
3058:	learn: 0.0142483	test: 0.0607205	best: 0.0607152 (3053)	total: 55.6s	remaining: 35.3s
3059:	learn: 0.0142446	test: 0.0607196	best: 0.0607152 (3053)	total: 55.6s	remaining: 35.3s
3060:	learn: 0.0142418	test: 0.0607179	best: 0.0607152 (3053)	total: 55.6s	remaining: 35.3s
3061:	learn: 0.0142393	test: 0.0607275	best: 0.0607152 (3053)	total: 55.7s	remaining: 35.2s
3062:	learn: 0.0142375	test: 0.0607269	best: 0.0607152 (3053)	total: 55.7s	remaining: 35.2s
3063:	learn: 0.0142338	test: 0.0607261	best: 0.0607152 (3053)	total: 55.7s	remai

In [ ]:
y_pred = model.predict_proba(x_test.drop(target, axis=1))

In [ ]:
res = y_pred[:,1]

In [ ]:
for x in res[:10]:
    print("{:f}".format(x))

In [ ]:
x_test.Stage.head(10)

In [ ]:
fuck = [x for x in res if x >= 0.025 and x <= 0.95]
len(fuck)

In [ ]:
len(fuck)/float(len(res)) * 100

In [ ]:
pred = pd.DataFrame(res, index = x_test.index, columns = ["Prediction"])

In [ ]:
x_test_pred = x_test.join(pred)
answer = x_test_pred.groupby("Opportunity_ID")["Stage","Prediction"].mean()

In [ ]:
log_loss(answer["Stage"], answer["Prediction"])

In [ ]:
predictors = [x for x in x_train.drop(target, axis = 1).columns]
feat_imp = pd.Series(model.feature_importances_, predictors).sort_values(ascending = False)
feat_imp = feat_imp[0:28]
plt.rcParams['figure.figsize'] = 28, 5
feat_imp.plot(kind = 'bar', title = 'Feature Importance')

In [ ]:
bad = x_test_pred[(x_test_pred.Prediction > 0.5) & (x_test_pred.Stage == 0) | (x_test_pred.Prediction < 0.5) & (x_test_pred.Stage == 1)]

In [ ]:
bad.Stage.value_counts()

In [ ]:
validation_file = "Validacion_ECI_2020.csv"
vali = pd.read_csv(validation_file)
validation = preprocess(vali)
leak = ["Opportunity_ID", "Sales_Contract_No"]
pred = model.predict_proba(validation.drop(leak, axis = 1))[:,1]

# agrupo por Opportunity_ID para dar una sola prediccion por solicitud

pred = pd.DataFrame(pred, index = validation.index, columns = ["Prediction"])
validation = validation.join(pred)

answer = pd.DataFrame(validation.groupby("Opportunity_ID", as_index = False)["Prediction"].mean())

In [ ]:
prev = pd.read_csv("acceptable/submission_23.csv", names=["Opportunity_ID", "Prediction"])
prev["Prediction"].corr(answer["Prediction"])

In [ ]:
prev = pd.read_csv("acceptable/submission_11.csv")
prev["Prediction"].corr(answer["Prediction"])

In [ ]:
#answer.to_csv("acceptable/submission_29.csv", header = False, index = False)